
*   NumPy: For numerical operations.
*   Pandas: For handling datasets.

*   Scikit-learn: For machine learning operations, including dataset loading, model training, and evaluation.








In [3]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



## **2. Probability Formula Used**
### **Bayes' Theorem**
The Naïve Bayes classifier is based on **Bayes’ Theorem**, which is given by:

\[
P(Class | Features) = \frac{P(Features | Class) \times P(Class)}{P(Features)}
\]

Where:
- \( P(Class | Features) \) = Posterior probability (probability of a class given the features).
- \( P(Features | Class) \) = Likelihood (probability of features given a class).
- \( P(Class) \) = Prior probability of the class.
- \( P(Features) \) = Evidence (probability of the features occurring).

### **Gaussian Naïve Bayes Assumption**
If the features follow a **normal (Gaussian) distribution**, the probability of a feature given a class is calculated as:

\[
P(x | Class) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x - \mu)^2}{2\sigma^2}}
\]

Where:
- \( x \) = Feature value
- \( \mu \) = Mean of the feature in that class
- \( \sigma^2 \) = Variance of the feature in that class

This assumption allows us to compute the probability of a feature belonging to a class using only its **mean and variance**.

### **Final Decision Rule**
For classification, we calculate the posterior probability for each class and select the class with the **highest probability**:

\[
Class = \arg\max P(Class | Features)
\]


## **1. Understanding the Formula**
The Naïve Bayes classifier is based on **Bayes' Theorem**, which is used to calculate the probability of a class given certain features.

### **Bayes' Theorem:**
\[
P(Class | Features) = \frac{P(Features | Class) \times P(Class)}{P(Features)}
\]

Where:
- \( P(Class | Features) \) = **Posterior probability**, i.e., the probability of a class given the features.
- \( P(Features | Class) \) = **Likelihood**, i.e., the probability of the features given a class.
- \( P(Class) \) = **Prior probability** of the class (how often the class appears in the dataset).
- \( P(Features) \) = **Evidence**, i.e., the probability of the features occurring.

### **Naïve Bayes Assumption:**
- The algorithm assumes that **features are independent**, meaning each feature contributes to the probability **individually**.
- This simplifies calculations and makes Naïve Bayes computationally efficient.

### **Gaussian Naïve Bayes (For Continuous Data)**
If the features are continuous (numerical), Naïve Bayes assumes they follow a **Gaussian (Normal) Distribution**:

\[
P(x | Class) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x - \mu)^2}{2\sigma^2}}
\]

Where:
- \( x \) = Feature value
- \( \mu \) = Mean of the feature in that class
- \( \sigma^2 \) = Variance of the feature in that class

### **Classification Decision Rule**
To classify a new sample, we calculate the probability for each class and assign the class with the **highest probability**:

\[
Class = \arg\max P(Class | Features)
\]

This means the class with the highest computed probability is chosen as the final prediction.


In [170]:
class NaiveBayesClassifier:
    def fit(self, X, y):
        """ Train the Naïve Bayes model by computing priors, means, and variances. """
        self.classes = np.unique(y)  # Get unique class labels
        self.means = {}
        self.variances = {}
        self.priors = {}

        for c in self.classes:
            X_c = X[y == c]  # Get all samples belonging to class c
            self.means[c] = np.mean(X_c, axis=0)  # Compute mean per feature
            self.variances[c] = np.var(X_c, axis=0)  # Compute variance per feature
            self.priors[c] = X_c.shape[0] / X.shape[0]  # Compute prior probability

    def _gaussian_probability(self, x, mean, var):
        """ Compute Gaussian probability density function """
        eps = 1e-6  # Small constant to prevent division by zero
        coeff = 1 / np.sqrt(2 * np.pi * var + eps)
        exponent = np.exp(-((x - mean) ** 2) / (2 * var + eps))
        return coeff * exponent

    def predict(self, X):
        """ Predict the class for each sample in X """
        predictions = [self._predict_sample(x) for x in X]
        return np.array(predictions)

    def _predict_sample(self, x):
        """ Compute probability for each class and return the most probable class """
        posteriors = []

        for c in self.classes:
            prior = np.log(self.priors[c])  # Log prior probability
            likelihoods = np.sum(np.log(self._gaussian_probability(x, self.means[c], self.variances[c])))
            posterior = prior + likelihoods  # Compute log-posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]  # Return class with highest probability

**Types of Naïve Bayes:**
* GaussianNB: Assumes features are normally distributed (used in our case).
* MultinomialNB: Used for text classification (e.g., spam filtering).
* BernoulliNB: Used for binary/boolean features.

In [173]:
# Load dataset
df = pd.read_csv('../Datasets/titanic_dataset.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [152]:
target = 'Survived'
df['Sex'] = df['Sex'].map({'male':1, 'female':0})
df['Embarked'] = df['Embarked'].map({'S':1, 'C':2, 'Q':3})
X = df[['Pclass']].values
y = df[target]

In [154]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [156]:
# Train Naïve Bayes classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X_train, y_train)

In [158]:
# Make predictions
y_pred = nb_classifier.predict(X_test)

In [168]:
# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.70


In [175]:
# print("Classification Report:")
# print(classification_report(y_test, y_pred, target_names=df[target]))